In [2]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.signal import resample, resample_poly
import struct
import imageio
import collections
import pickle
import re
import requests
import pandas as pd
from pathlib2 import Path
import pretty_errors
from filter_BU import filt_B
import my_pyrotd
from awp_processing import awp
from post_processing.la_habra import *

import seaborn as sns
import matplotlib.pyplot as plt

np.errstate(divide='ignore')
#%config InlineBackend.figure_format = 'retina'
# %matplotlib inline
%load_ext autoreload
%autoreload 2


<Figure size 432x288 with 0 Axes>

## functions

In [3]:
from obspy.signal.tf_misfit import plot_tfr, em, pm, tem, tpm, fem, fpm, tfem, tfpm

def resize(vel, vel_rec, dt=None):
    length = int(len(vel['t']) * vel['dt'] // dt)
    length_rec = int(len(vel_rec['t']) * vel_rec['dt'] // dt)
    #print(f'Length of time steps = {length}')
    if length > len(vel['t']):
        print(f"Upsample may introduce alias. Use larger dt_sample instead.\n")
        return
    for comp in 'XYZ':
        if length > length_rec:     
            vel[comp] = vel[comp][vel['t'] <= vel_rec['t'][-1]]
            vel[comp] = resample(vel[comp], length_rec)
            vel_rec[comp] = resample(vel_rec[comp], length_rec)
        else:
            vel_rec[comp] = vel_rec[comp][vel_rec['t'] <= vel['t'][-1]]
            vel_rec[comp] = resample(vel_rec[comp], length)
            vel[comp] = resample(vel[comp], length)
            
   
def plot_obspy_tf_misfit(misfit, comps='XYZ', left=0.1, bottom=0.1,
                    h_1=0.2, h_2=0.125, h_3=0.2, w_1=0.2, w_2=0.6, w_cb=0.01,
                    d_cb=0.0, show=True, plot_args=['k', 'r', 'b'], ylim=0.,
                    clim=0., cmap='RdBu_r', fmin=0.15, fmax=5, dpi=300):
    from matplotlib.ticker import NullFormatter
    figs = []
    t = misfit['t']
    f = misfit['f']
    ntr = len(comps)
    for itr, comp in enumerate(comps):
        fig = plt.figure(dpi=dpi)
        data = misfit[comp]
        
        # plot signals
        ax_sig = fig.add_axes([left + w_1, bottom + h_2 + h_3, w_2, h_1])
        ax_sig.plot(t, data['rec'], plot_args[0], label=f'data, max={100 * np.max(data["rec"]):.3f} cm/s')
        ax_sig.plot(t, data['syn'], plot_args[1], label=f'syn, max={100 * np.max(data["syn"]):.3f} cm/s')
        ax_sig.legend(loc=1, ncol=2)

        # plot TEM
        if 'tem' in data:
            ax_tem = fig.add_axes([left + w_1, bottom + h_1 + h_2 + h_3, w_2, h_2])
            ax_tem.plot(t, data['tem'], plot_args[2])

        # plot TFEM
        if 'tfem' in data:
            ax_tfem = fig.add_axes([left + w_1, bottom + h_1 + 2 * h_2 + h_3, w_2,
                                    h_3])

            img_tfem = ax_tfem.pcolormesh(t, f, data['tfem'], cmap=cmap)
            img_tfem.set_rasterized(True)
            ax_tfem.set_yscale("log")
            ax_tfem.set_ylim(fmin, fmax)

        # plot FEM
        if 'fem' in data:
            ax_fem = fig.add_axes([left, bottom + h_1 + 2 * h_2 + h_3, w_1, h_3])
            ax_fem.semilogy(data['fem'], f, plot_args[2])
            ax_fem.set_ylim(fmin, fmax)

        # plot TPM
        if 'tpm' in data:
            ax_tpm = fig.add_axes([left + w_1, bottom, w_2, h_2])
            ax_tpm.plot(t, data['tpm'], plot_args[2])

        # plot TFPM
        if 'tfpm' in data:
            ax_tfpm = fig.add_axes([left + w_1, bottom + h_2, w_2, h_3])

            img_tfpm = ax_tfpm.pcolormesh(t, f, data['tfpm'], cmap=cmap)
            img_tfpm.set_rasterized(True)
            ax_tfpm.set_yscale("log")
            ax_tfpm.set_ylim(f[0], f[-1])

        # add colorbars
        ax_cb_tfpm = fig.add_axes([left + w_1 + w_2 + d_cb + w_cb, bottom,
                                   w_cb, h_2 + h_3])
        fig.colorbar(img_tfpm, cax=ax_cb_tfpm)

        # plot FPM
        if 'fpm' in data:
            ax_fpm = fig.add_axes([left, bottom + h_2, w_1, h_3])
            ax_fpm.semilogy(data['fpm'], f, plot_args[2])
            ax_fpm.set_ylim(fmin, fmax)

        # set limits
        ylim_sig = np.max([np.abs(data['rec']).max(), np.abs(data['syn']).max()]) * 2
        ax_sig.set_ylim(-ylim_sig, ylim_sig)

        if ylim == 0.:
            ylim = np.max([np.abs(data[key]).max() for key in ['tem', 'tpm', 'fem', 'fpm'] \
                           if key in data]) * 1.1
    
        ax_tem.set_ylim(-ylim, ylim)
        ax_fem.set_xlim(-ylim, ylim)
        ax_tpm.set_ylim(-ylim, ylim)
        ax_fpm.set_xlim(-ylim, ylim)

        ax_sig.set_xlim(t[0], t[-1])
        ax_tem.set_xlim(t[0], t[-1])
        ax_tpm.set_xlim(t[0], t[-1])

        if clim == 0.:
            clim = np.max([np.abs(data['tfem']).max(), np.abs(data['tfpm']).max()])

        img_tfpm.set_clim(-clim, clim)
        img_tfem.set_clim(-clim, clim)

        # add text box for EM + PM
        textstr = f"{comp}-component\nEM = {data['em']: .2f}\nPM = {data['pm']: .2f}"
        props = dict(boxstyle='round', facecolor='white')
        ax_sig.text(-0.3, 0.5, textstr, transform=ax_sig.transAxes,
                    verticalalignment='center', horizontalalignment='left',
                    bbox=props)

        ax_tpm.set_xlabel('time (s)')
        ax_fem.set_ylabel('frequency (Hz)')
        ax_fpm.set_ylabel('frequency (Hz)')

        # add text boxes
        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        ax_tfem.text(0.95, 0.85, 'TFEM', transform=ax_tfem.transAxes,
                     verticalalignment='top', horizontalalignment='right',
                     bbox=props)
        ax_tfpm.text(0.95, 0.85, 'TFPM', transform=ax_tfpm.transAxes,
                     verticalalignment='top', horizontalalignment='right',
                     bbox=props)
        ax_tem.text(0.95, 0.75, 'TEM', transform=ax_tem.transAxes,
                    verticalalignment='top', horizontalalignment='right',
                    bbox=props)
        ax_tpm.text(0.95, 0.75, 'TPM', transform=ax_tpm.transAxes,
                    verticalalignment='top', horizontalalignment='right',
                    bbox=props)
        ax_fem.text(0.9, 0.85, 'FEM', transform=ax_fem.transAxes,
                    verticalalignment='top', horizontalalignment='right',
                    bbox=props)
        ax_fpm.text(0.9, 0.85, 'FPM', transform=ax_fpm.transAxes,
                    verticalalignment='top', horizontalalignment='right',
                    bbox=props)

        # remove axis labels
        ax_tfpm.xaxis.set_major_formatter(NullFormatter())
        ax_tfem.xaxis.set_major_formatter(NullFormatter())
        ax_tem.xaxis.set_major_formatter(NullFormatter())
        ax_sig.xaxis.set_major_formatter(NullFormatter())
        ax_tfpm.yaxis.set_major_formatter(NullFormatter())
        ax_tfem.yaxis.set_major_formatter(NullFormatter())

        figs.append(fig)

    if show:
        plt.show()
    else:
        if ntr == 1:
            return figs[0]
        else:
            return figs
        
        
def comp_obspy_tf_misfit(model, site_name, dt, comps='XYZ', 
                         fmin=0.15, fmax=5, nf=128, vel=None, vel_rec=None, plot=False):
    if vel is None:
        with open('results/vel_syn.pickle', 'rb') as fid:
            vel_syn = pickle.load(fid)
        vel = vel_syn[model][site_name]
        vel_rec = vel_syn["rec"][site_name]
        
    resize(vel, vel_rec, dt)
    res = {}
    res['dt'] = dt
    res['f'] = np.logspace(np.log10(fmin), np.log10(fmax), nf)
    
    for comp in comps:
        res[comp] = {}
        res[comp]['syn'] = vel[comp]
        res[comp]['rec'] = vel_rec[comp]
        res[comp]['em'] = em(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['pm'] = pm(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['tfem'] = tfem(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['tfpm'] = tfpm(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['tem'] = tem(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['tpm'] = tpm(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['fem'] = fem(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['fpm'] = fpm(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
    res['t'] = np.arange(res[comp]['tem'].shape[-1]) * dt
    if plot:
        plot_obspy_tf_misfit(res, comps=comps)
    return res
        
    
    
def comp_obspy_tf_misfits(models, dt, comps='XYZ', fmin=0.15, fmax=5, nf=128):
    misfit = {}
    with open('results/vel_syn.pickle', 'rb') as fid:
        vel_syn = pickle.load(fid)
        
    for model in models:
        misfit[model] = {}
        vel = vel_syn[model]
        vel_rec = vel_syn['rec']
        for site_name in vel_syn[model].keys():
            print(f'{model}: {site_name}')
            misfit[model][site_name] = comp_obspy_tf_misfit( \
                    model, site_name, dt, comps=comps, fmin=fmin, fmax=fmax, 
                    nf=nf, vel=vel[site_name], vel_rec=vel_rec[site_name])
    return misfit

def comp_obspy_tf_misfit_short(model, site_name, dt, comps='XYZ', 
                         fmin=0.15, fmax=5, nf=128, vel=None, vel_rec=None, plot=False):
    if vel is None:
        with open('results/vel_syn.pickle', 'rb') as fid:
            vel_syn = pickle.load(fid)
        vel = vel_syn[model][site_name]
        vel_rec = vel_syn["rec"][site_name]
        
    resize(vel, vel_rec, dt)
    res = {}
    res['dt'] = dt
    res['f'] = np.logspace(np.log10(fmin), np.log10(fmax), nf)
    
    for comp in comps:
        res[comp] = {}
        res[comp]['syn'] = vel[comp]
        res[comp]['rec'] = vel_rec[comp]
        res[comp]['em'] = em(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['pm'] = pm(vel[comp], vel_rec[comp], dt, fmin, fmax, nf)
        res[comp]['eg'] = 10 * np.exp(-abs(res[comp]['em']))
        res[comp]['pg'] = 10 * (1 - np.abs(res[comp]['pm']))
    res['t'] = np.arange(len(res[comp]['syn'])) * dt
    return res


def comp_obspy_tf_misfits_short(models, dt, comps='XYZ', fmin=0.15, fmax=5, nf=128):
    with open('results/vel_syn.pickle', 'rb') as fid:
        vel_syn = pickle.load(fid)
        
    misfit = {}
    for model in models:
        misfit[model] = {}
        vel = vel_syn[model]
        vel_rec = vel_syn['rec']
        for i, site_name in enumerate(vel_syn[model].keys()):
            if i % 50 == 0:
                print(f'{model}: {site_name}')
            tmp = comp_obspy_tf_misfit_short( \
                    model, site_name, dt, comps=comps, fmin=fmin, fmax=fmax, 
                    nf=nf, vel=vel[site_name], vel_rec=vel_rec[site_name])
            out = []
            for i, comp in enumerate(comps):
                out.append(tmp[comp]['em'])  
                out.append(tmp[comp]['pm'])  
            for i, comp in enumerate(comps):
                out.append(tmp[comp]['eg'])  
                out.append(tmp[comp]['pg'])
            misfit[model][site_name] = np.array(out, dtype='float32').reshape(6, 2)
    return misfit

In [4]:
def distance(lon1, lat1, lon2=-117.932587, lat2=33.918633):
    lat1, lon1, lat2, lon2 = np.radians((lat1, lon1, lat2, lon2))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    d = 0.5 - np.cos(dlat) / 2 + np.cos(lat1) * np.cos(lat2)  * (1 - np.cos(dlon)) / 2
    return 12742 * np.arcsin(np.sqrt(d))

In [5]:
def read_pickle(fname):
    return  pickle.loads(Path(fname).read_bytes())

def write_pickle(data, fname):
    with open(fname, 'wb') as fid:
        pickle.dump(data, fid, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
def read_topo(fname, mx, my):
    """Return topog (my, mx)"""
    pad = 8
    with open(fname, 'rb') as fout:
        mx, my, pad = np.frombuffer(fout.read(12), dtype='int32')
        topo = np.frombuffer(fout.read((mx + 2 * pad) * (my + 2 * pad) * 4),
                             dtype='float32').reshape(mx + 2 * pad, my + 2 * pad).T
        return topo[pad : -pad, pad : -pad]

In [7]:
# band pass filter
lowf, highf = 0.15, 5
osc_freqs = np.concatenate((np.linspace(0.1, 10, 100),
                        [1/3, 2, 3, 4, 5]))
osc_freqs = sorted(osc_freqs)
fqs = [2, 3, 4, 5]


mx, my = 19440, 14904
dh = 8
tmax, dt, tskip, wstep, nfile = read_param("")
tpad = tmax + 5  # Padding 5 seconds when using bbs preparing
nd = 500  # avoid abc boudnary, which is at most 80 * 3 = 240
dt = dt * tskip
nt = int(tmax / dt)
fs = 1 / dt

# Topography
if "topography" not in locals():
    topography = read_topo('topography.bin', mx, my)
#     topo_grad = np.gradient(topography, dh)
#     topo_grad = np.sqrt(topo_grad[0] ** 2 + topo_grad[1] ** 2)
    
# sites of recordings
# url_rec = 'http://hypocenter.usc.edu/bbp/highf/data/2020-08-10-data-processed/'
# r = requests.get(url_rec)
# rec_sites = re.findall('(?<=">p-).+(?=\\.V2.vel)', r.text)

# orig_sites = np.genfromtxt('stat_name_idx.txt', delimiter=" ", dtype="S8, i4, i4")
# seems Fabio doesn't remove "_" in site name now
if os.path.isfile('results/syn_sites.pickle'):
    with open('results/syn_sites.pickle', 'rb') as fid:
        syn_sites = pickle.load(fid)
else:
    syn_sites = [(name.decode('UTF-8').replace('_', ''), ix, iy) for name, ix, iy in orig_sites \
             if nd < ix < mx - nd and nd < iy < my - nd] 
    with open('results/syn_sites.pickle', 'wb') as fid:
        pickle.dump(syn_sites, fid, protocol=pickle.HIGHEST_PROTOCOL)

# Source index 
if 1:# and 'srcidx' not in locals():
#     grids = np.fromfile('surf.grid', dtype='float64').reshape(my, mx, 3)[:, :, :2]
#     grids_squeeze = np.reshape(grids, (-1, 2))
    nsrcx = nsrcz = 125
    src_lonlat = [-117.932587, 33.918633]
    src_idx = np.genfromtxt('fault_idx.txt', dtype='int').reshape(nsrcz, nsrcx, 3)
    srcidx = [int(x) for x in (np.mean(src_idx, axis=(0, 1)))]  # (ix, iy, iz)
    
    print(srcidx)
    # from scipy import spatial
    # kdtree = spatial.cKDTree(grids_squeeze)
    # query_idx = np.unravel_index(kdtree.query(src_lonlat)[1], (my, mx))  # (iy, ix)
    # srcidx = query_idx[::-1] + (srcidx[-1],)  #(ix, iy, iz)
    # srcidx = [4037, 2732, 709]
#     print(f'Interpolated source lon/lat: {grids[srcidx[1], srcidx[0]]}\n', 
#           f'Close to records? {np.isclose(grids[srcidx[1], srcidx[0]], src_lonlat, atol=5e-5)}')
#     del grids

site_latlon = np.genfromtxt('la_habra_large_statlist_070120.txt', usecols=[0,1,2,3], dtype="S8, f, f, f")
site_dist = {}
site_vs30 = {}
site_elev = {}
site_idx = {}
for i in range(len(site_latlon)):
    site_name = site_latlon[i][0].decode('UTF-8').replace('_', '')
    site_idx[site_name] = (syn_sites[i][1], syn_sites[i][2])
    site_dist[site_name] = np.sqrt((srcidx[-1] * dh / 1000) ** 2 +  \
                           distance(site_latlon[i][1], site_latlon[i][2]) ** 2)
    site_vs30[site_name] = site_latlon[i][3]
    site_elev[site_name] = topography[syn_sites[i][2], syn_sites[i][1]]

sorted_site_elev = [(k, v) for k, v in sorted(site_elev.items(), key=lambda x: x[1])]
sorted_site_dist = [(k, v) for k, v in sorted(site_dist.items(), key=lambda x: x[1])]


  
# # VS30 is at the approxmately 5th layer; skip = 4
# from scipy.stats import hmean
# vs30 = np.fromfile('mesh', dtype='float32', count=mx * my * 3 * 3).reshape(3, my, mx, 3)[:, :, :, 1]
# vs30 = hmean(vs30, axis=0)
vs = np.fromfile('mesh_large_8m_orig.bin_0', dtype='float32', count=mx * my * 3).reshape(my, mx, 3)[:, :, 1]


[9681, 5086, 709]


In [8]:
ds = []
for i in range(len(site_latlon)):
    d = {}
    site_name = site_latlon[i][0].decode('UTF-8').replace('_', '')
    d["site_name"] = site_name 
    d["lon"] = site_latlon[i][1]
    d["lat"] = site_latlon[i][2]
    d["idx_x"] = syn_sites[i][1]
    d["idx_y"] = syn_sites[i][2]
    d["vs30"] = site_latlon[i][3]
    d["vs"] = vs[syn_sites[i][2], syn_sites[i][1]]
    d["elevation"] = topography[syn_sites[i][2], syn_sites[i][1]]
    d["rhyp"] = site_dist[site_name]
    ds.append(d)
df_sites = pd.DataFrame(ds, index=None)
df_sites.set_index('site_name', drop=True, inplace=True)
df_sites.sort_values("rhyp", inplace=True)
#df_sites.reset_index(drop=False, inplace=True)
#df_sites.to_csv("results/df_sites.csv", index=False)

# or row.idx_x < srcidx[0]
df_sites['west'] = df_sites.apply(lambda row: "west" if row.lon < src_lonlat[0] \
                                 else "east", axis=1)
site_info = df_sites.columns.tolist()
%store df_sites
df_sites.head()
del vs

Stored 'df_sites' (DataFrame)


In [9]:
left, right, bot, top = -118.5, -117.23, 33.6, 34.35

labels = {'arias': r'Arias (cm/s)', 'pga': r'PGA (cm/s@+2@+)', 'pgv': r'PGV (cm/s)',
          'ener': r'ENER (cm/s@+2@+)', 'dur': 'DUR (s)'}
metrics = list(labels.keys())

%store -r model_dict
model_id = {k: i for i, k in enumerate(model_dict)}

## Metric preparation

### Velocity

In [10]:
vel_syn = collections.defaultdict(dict)


models = ['dhyp0.50_s1485839278_q100f00_orig_vs200',
          'dhyp0.50_s1485839278_q100f00_orig_vs500',
          'dhyp1.50_s372823598_q100f00_orig_vs200',
          'dhyp1.00_s387100462_q100f00_orig_vs200',
          'dhyp0.50_s1485839278_q100f00_s05h005l100_vs200',
          'dhyp0.50_s1485839278_q100f00_s05h005l500_vs200',
          'dhyp0.50_s1485839278_q100f00_s10h005l500_vs200',
          'rec']

with open('results/vel_syn.pickle', 'rb') as fid:
    vel_syn = pickle.load(fid)


# models = ['dhyp0.50_s1485839278_q100f00_s10h005l100_vs200']
# for model in models:
#     if model in vel_syn.keys():
#         continue
#     try:
#         with open(Path(model, 'vel_sites.pickle'), 'rb') as fid:      
#             vel_syn[model] = pickle.load(fid) 
#             for k in vel_syn[model].keys():  # Each site
#                 vel_syn[model][k] = rotate(vel_syn[model][k], -39.9)
#                 vel_syn[model][k] = prepare_bbpvel(vel_syn[model][k], tmax)

#     except:
#         print("No model found: ", model)
#         with open(f'results/vel_{model}.pickle', 'rb') as fid:
#             vel_syn[model] = pickle.load(fid)
            
# with open(f'results/vel_rec.pickle', 'rb') as fid:
#     vel_syn['rec'] = pickle.load(fid)
# # for k in vel_syn['rec'].keys():  # Each site
# #     vel_syn['rec'][k] = prepare_bbpvel(vel_syn['rec'][k], tmax, shift=vel_syn['rec'][k]['shift'], 
# #                                        dt=vel_syn['dhyp0.50_s1485839278_q100f00_orig_vs200'][k]['dt'])

# with open(Path("../la_habra_large_gpu_abc50/topo_q100f08_sh005l100_vs30_vs500", 'vel_sites.pickle'), 'rb') as fid:      
#     tmp = pickle.load(fid) 
#     for k in tmp.keys():  # Each site
#         tmp[k] = rotate(tmp[k], -39.9)
#         tmp[k] = prepare_bbpvel(tmp[k], tmax)

# vel_syn['topo_q100f08_sh005l100_vs30_vs500'] = tmp

# with open('results/vel_syn.pickle', 'wb') as fid:
#     pickle.dump(vel_syn, fid, protocol=pickle.HIGHEST_PROTOCOL) 

In [23]:
# # Read bbp-format recordings

# def read_bbp(texts):
#     res = {}
#     data = np.genfromtxt(texts)
#     res['t'] = data[:, 0]
#     res['dt'] = data[1, 0] - data[0, 0]
#     for i, comp in enumerate('YXZ', 1):
#         res[comp] = dat[:, i] / 100  # cm/s2 --> m/s2
#     return res

# vel_rec = collections.defaultdict(dict)
# for i, isite in enumerate(orig_sites):
#     site_name = isite[0].decode('UTF-8')
#     r = requests.get(f'{url_rec}/p-{site_name}.V2.vel.bbp')
#     if r.status_code != 200:
#         print(f"the site {site_name} not found on the server")
#         continue
#     vel_rec[site_name.replace('_', '')] = read_bbp(r.text.split('\n'))
# vel_syn['rec'] = vel_rec

# # End reading


# with open('results/vel_syn.pickle', 'wb') as fid:
#     pickle.dump(vel_syn, fid, protocol=pickle.HIGHEST_PROTOCOL) 

In [28]:
model = 'topo_q100f06_sh005l100_1000vs30erf100_cap_vs500'
with open(Path(f"../la_habra_large_gpu_abc50/{model}", 'vel_sites.pickle'), 'rb') as fid:      
    tmp = pickle.load(fid) 
    for k in tmp.keys():  # Each site
        tmp[k] = rotate(tmp[k], -39.9)
        tmp[k] = prepare_bbpvel(tmp[k], tmax)

vel_syn[model] = tmp
with open('results/vel_syn.pickle', 'wb') as fid:
    pickle.dump(vel_syn, fid, protocol=pickle.HIGHEST_PROTOCOL) 


### SA

In [42]:
psa_syn = read_pickle('results/psa_syn.pickle')

# models = ['dhyp0.50_s1485839278_q100f00_orig_vs200',
#           'dhyp0.50_s1485839278_q100f00_orig_vs500',
#           'dhyp1.50_s372823598_q100f00_orig_vs200',
#           'dhyp1.00_s387100462_q100f00_orig_vs200',
#           'dhyp0.50_s1485839278_q100f00_s05h005l100_vs200',
#           'dhyp0.50_s1485839278_q100f00_s05h005l500_vs200',
#           'dhyp0.50_s1485839278_q100f00_s10h005l500_vs200',
#           'rec',
#          ]
models = [
    #'topo_q100f08_sh005l100_vs30_vs500',
#     'topo_q50t07f08_vs30_vs500',
#     'topo_q50f08_vs30erf100_vs500',
#     'topo_q50f08_vs30erf200_vs500',
#     'topo_q100f06_1000vs30erf100_cap_vs500',
#     'topo_q50f08_orig_vs500',
#     'topo_q100f06_orig_vs500',
#     'topo_q100f06_sh005l100_1000vs30erf100_cap_vs500',
    'topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500',
]
psa_syn = pick_psa(mx, my, models, force_update=True,
                   osc_freqs=osc_freqs, syn_sites=syn_sites)


Models queried!
['topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500']

Gathering psa_syn for CE12919, 0/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13066, 1/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13067, 2/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13068, 3/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13069, 4/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13079, 5/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13080, 6/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13096, 7/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13098, 8/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13099, 9/259
: topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Gathering psa_syn for CE13100, 10/259
: topo_q100f

### tf_misfit

In [12]:
# Run this one
# EM/PM/EG/PG only tf_misfit

dt = 0.04
try:
    tf_misfit = pickle.loads(Path('results/tf_misfit.pickle').read_bytes())
except:
    tf_misfit = {}
    
models = ['dhyp0.50_s1485839278_q100f00_orig_vs200',
          'dhyp0.50_s1485839278_q100f00_orig_vs500',
          'dhyp1.50_s372823598_q100f00_orig_vs200',
          'dhyp1.00_s387100462_q100f00_orig_vs200',
          'dhyp0.50_s1485839278_q100f00_s05h005l100_vs200',
          'dhyp0.50_s1485839278_q100f00_s05h005l500_vs200',
          'dhyp0.50_s1485839278_q100f00_s10h005l500_vs200']
models = [
    #'topo_q50f08_s05h005l100_vs500',
    #'topo_q100f00_orig_vs500',
    #'topo_q100f06_s05h005l100_vs500',
#     'topo_q100f08_sh005l100_vs30_vs500',
#     'topo_q100f08_sh005l500_vs30_vs500',
#     'topo_q50t07f08_vs30_vs500',
    #'dhyp0.50_s1485839278_q100f00_s10h005l100_vs200',
#     'topo_q50f08_vs30erf100_vs500',
#     'topo_q50f08_vs30erf200_vs500',
#     'topo_q100f06_1000vs30erf100_cap_vs500',
#     'topo_q50f08_orig_vs500',
#     'topo_q100f06_orig_vs500',
#     'topo_q100f06_sh005l100_1000vs30erf100_cap_vs500',
    'topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500',
]
tmp = {}
for f in [(0.15, 2.5), (0.15, 5), (2.5, 5), (0.15, 1), (0.15, 4)]:
    if f not in tf_misfit:
        tf_misfit[f] = {}
#     tf_misfit[f].pop('topo_q50t07f08_vs30_vs500')
    models = [model for model in models if model not in tf_misfit[f]]
    tmp = comp_obspy_tf_misfits_short(models, dt, fmin=f[0], fmax=f[1])
    for model in models:
        print(model)
        tf_misfit[f][model] = tmp[model]
print(tf_misfit[(0.15, 2.5)].keys())


# # force rec tf_misfits
# rec_tf_misfit = np.zeros((6, 2))
# rec_tf_misfit[3:,:] = 10.
# for f in tf_misfit.keys():
#     tf_misfit[f]['rec'] = {}
#     for site_name in tf_misfit[f][models[0]].keys():
#         tf_misfit[f]['rec'][site_name] = rec_tf_misfit.copy()
        
with open('results/tf_misfit.pickle', 'wb') as fid:
    pickle.dump(tf_misfit, fid, protocol=pickle.HIGHEST_PROTOCOL)

topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE12919
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE14004
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE23075
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE24592
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CIMLS
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CISTG
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE12919
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE14004
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE23075
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE24592
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CIMLS
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CISTG
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE12919
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE14004
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500: CE23075
topo_q100f06_sh005l1000_1000vs30erf100_c

### Metrics & GOF

In [13]:
# models = ['dhyp0.50_s1485839278_q100f00_orig_vs200',
#           'dhyp0.50_s1485839278_q100f00_orig_vs500',
#           'dhyp1.50_s372823598_q100f00_orig_vs200',
#           'dhyp1.00_s387100462_q100f00_orig_vs200',
#           'dhyp0.50_s1485839278_q100f00_s05h005l100_vs200',
#           'dhyp0.50_s1485839278_q100f00_s05h005l500_vs200',
#           'dhyp0.50_s1485839278_q100f00_s10h005l500_vs200',
#          ]
# models = [
#     'topo_q50f08_s05h005l100_vs500',
#     'topo_q100f00_orig_vs500',
#     'topo_q100f06_s05h005l100_vs500',
# ]
models = [
#     'topo_q100f08_sh005l100_vs30_vs500',
#     'topo_q100f08_sh005l500_vs30_vs500',
#     'topo_q50t07f08_vs30_vs500',
#     'topo_q50f08_vs30erf100_vs500',
#     'topo_q50f08_vs30erf200_vs500',
#     'topo_q100f06_cap_vs500',
#     'topo_q100f06_1000vs30erf100_cap_vs500',
#     'topo_q50f08_orig_vs500',
#     'topo_q100f06_orig_vs500',
#      'topo_q100f06_sh005l100_1000vs30erf100_cap_vs500',
    'topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500'
]

comp_metrics(models, vel_syn, lowcut=0.15, highcut=[0, 1, 2.5, 4, 5],
             tmax=tmax, force_update=1, save=True) 
comp_metrics(models, vel_syn, lowcut=2.5, highcut=[5], tmax=tmax,
             force_update=1, save=True)  

met = pickle.loads(Path('results/metrics.pickle').read_bytes())
# models = list(met[(0.15, 1)].keys())

gof = comp_GOF([0, 1, 2.5, 4, 5, (2.5, 5)] , models, vs=site_vs30, 
    syn_sites=syn_sites, topography=topography, sx=srcidx[0], sy=srcidx[1],
        sz=srcidx[2], dh=0.008)


gof = pickle.loads(Path('results/gof.pickle').read_bytes())

Processing frequency band = (0.15, 0) Hz
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Processing frequency band = (0.15, 1) Hz
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Processing frequency band = (0.15, 2.5) Hz
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Processing frequency band = (0.15, 4) Hz
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Processing frequency band = (0.15, 5) Hz
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Processing frequency band = (2.5, 5) Hz
topo_q100f06_sh005l1000_1000vs30erf100_cap_vs500
Done (0.15, 0)Hz
Done (0.15, 1)Hz
Done (0.15, 2.5)Hz
Done (0.15, 4)Hz
Done (0.15, 5)Hz
Done (2.5, 5)Hz


In [15]:
# Merge previous results if not present
if 0:
    tmp = read_pickle("../la_habra_large_gpu_abc50/results/vel_syn.pickle")
    vel = pickle.loads(Path('results/vel_syn.pickle').read_bytes())
    if model not in vel:
        vel[model] = tmp[model]
    write_pickle(vel, 'results/vel_syn.pickle')

    tmp = read_pickle("../la_habra_large_gpu_abc50/results/psa_syn.pickle")
    psa = pickle.loads(Path('results/psa_syn.pickle').read_bytes())
    for model in tmp.keys():
        if model not in psa:
            psa[model] = tmp[model]
    write_pickle(psa, 'results/psa_syn.pickle')

    tmp = read_pickle("../la_habra_large_gpu_abc50/results/psax_syn.pickle")
    psa = pickle.loads(Path('results/psax_syn.pickle').read_bytes())
    for model in tmp.keys():
        if model not in psa:
            psa[model] = tmp[model]
    write_pickle(psa, 'results/psax_syn.pickle')

    tmp = read_pickle("../la_habra_large_gpu_abc50/results/psay_syn.pickle")
    psa = pickle.loads(Path('results/psay_syn.pickle').read_bytes())
    for model in tmp.keys():
        if model not in psa:
            psa[model] = tmp[model]
    write_pickle(psa, 'results/psay_syn.pickle')


    tmp = read_pickle("../la_habra_large_gpu_abc50/results/gof.pickle")
    gof = pickle.loads(Path('results/gof.pickle').read_bytes())
    for f in tmp.keys():
        for model in tmp[f].keys():
            if f in gof and model not in gof[f]:
                gof[f][model] = tmp[f][model]
    write_pickle(gof, 'results/gof.pickle')


    tmp = read_pickle("../la_habra_large_gpu_abc50/results/metrics.pickle")
    met = pickle.loads(Path('results/metrics.pickle').read_bytes())
    for f in tmp.keys():
        for model in tmp[f].keys():
            if f in met and model not in met[f]:
                met[f][model] = tmp[f][model]
    write_pickle(met, 'results/metrics.pickle')

    tmp = read_pickle("../la_habra_large_gpu_abc50/results/tf_misfit.pickle")
    tf_misfit = pickle.loads(Path('results/tf_misfit.pickle').read_bytes())
    for f in tmp.keys():
        for model in tmp[f].keys():
            if f in tf_misfit and model not in tf_misfit[f]:
                tf_misfit[f][model] = tmp[f][model]
    write_pickle(tf_misfit, 'results/tf_misfit.pickle')

    del tmp

In [14]:
## Single tf_misfit for 0.15 - 5 Hz, with all components, fem, fpm ...


# fmin, fmax = 0.15, 5
# models = ['dhyp0.50_s1485839278_q100f00_orig_vs200']
# obspy_tf_misfit = pickle.loads(Path('results/obspy_tf_misfit.pickle').read_bytes())

# # if os.path.isfile('results/obspy_tf_misfit.pickle'):
# #     obspy_tf_misfit = pickle.load(open('results/obspy_tf_misfit.pickle', 'rb'))
# # else:
# dt = 0.04
# for model in models:
#     if model not in obspy_tf_misfit:
#         obspy_tf_misfit[model] = comp_obspy_tf_misfits([model], dt).pop(model)

# #     with open('results/obspy_tf_misfit.pickle', 'wb') as fid:
# #         pickle.dump(obspy_tf_misfit, fid, protocol=pickle.HIGHEST_PROTOCOL)


### Temp

In [15]:
if 1:
    model_from = 'topo_q100f06_sh005l1000_cap_vs500'
    model_to = 'topo_q100f06_sh005l100_1000vs30erf100_cap_vs500'
#     vel_syn = read_pickle('results/vel_syn.pickle')
#     psa_syn = read_pickle('results/psa_syn.pickle')
#     psax_syn = read_pickle('results/psax_syn.pickle')
#     psay_syn = read_pickle('results/psay_syn.pickle')
#     gof = read_pickle('results/gof.pickle')
#     met = read_pickle('results/metrics.pickle')
    tf_misfit = read_pickle('results/tf_misfit.pickle')

#     vel_syn[model_to] = vel_syn.pop(model_from)
#     psa_syn[model_to] = psa_syn.pop(model_from)
#     psax_syn[model_to] = psax_syn.pop(model_from)
#     psay_syn[model_to] = psay_syn.pop(model_from)

#     for f in met:
#         met[f][model_to] = met[f].pop(model_from)
#         gof[f][model_to] = gof[f].pop(model_from)
#     for f in tf_misfit:
#         tf_misfit[f][model_to] = tf_misfit[f].pop(model_from)
    
#     for f in met:
#         met[f].pop(model_from)
#         gof[f].pop(model_from)
    for f in tf_misfit:
        tf_misfit[f].pop(model_from)

#     write_pickle(vel_syn, 'results/vel_syn.pickle')
#     write_pickle(psa_syn, 'results/psa_syn.pickle')
#     write_pickle(psax_syn, 'results/psax_syn.pickle')
#     write_pickle(psay_syn, 'results/psay_syn.pickle')
#     write_pickle(gof, 'results/gof.pickle')
#     write_pickle(met, 'results/metrics.pickle')
    write_pickle(tf_misfit, 'results/tf_misfit.pickle')

In [16]:
# Construct the DataFrame from met and gof

idx = pd.IndexSlice
met = pickle.loads(Path('results/metrics.pickle').read_bytes())
gof = pickle.loads(Path('results/gof.pickle').read_bytes())

df = pd.DataFrame.from_dict(
    {
        (f, model, site_name) : met[f][model][site_name]
             for f in sorted(met.keys())
             for model in met[f]
             for site_name in met[f][model].keys()
    },
    orient='index')
df = df.sort_index().sort_index(axis=1).unstack(0).unstack(0)

df_gof = pd.DataFrame.from_dict(
    {
        (f, model, site_name) : gof[f][model][site_name]
             for f in sorted(gof.keys())
             for model in gof[f]
             for site_name in gof[f][model].keys()
    },
    orient='index')
df_gof = df_gof.sort_index().sort_index(axis=1).unstack(0).unstack(0)
df_gof.drop(['rhypo', 'vs', 'elev'], axis=1, level=0, inplace=True)


# psa
psa = pickle.loads(Path('results/psa_syn.pickle').read_bytes())
freqs = pd.Series([1,2,3,3.5], name='freq')  # frequencies of PSA to query

index = pd.MultiIndex.from_product([
    ['psa'],
    freqs,
    df.columns.unique(2),
], names=df.columns.names)
df_psa = pd.DataFrame(index=df.index, columns=index)

for model in df_psa.columns.unique(2):  # model label
    osc_freqs = psa[model][df.index[0]].osc_freq
    for freq in df_psa.columns.unique(1):  # freq label
        idx_f = np.searchsorted(osc_freqs, freq)
        for site_name in df_psa.index:
            df_psa.loc[site_name, idx['psa', freq, model]] = \
                       psa[model][site_name].spec_accel[idx_f]
    
    
df = df.join(df_gof).join(df_psa)

# # Append site information from df_sites
# tmp = df_sites.copy()
# df[tmp.columns] = tmp


# # Sort by rhyp
# df.sort_values('rhyp', axis=0, inplace=True)
# df.sort_index(key = lambda x : df_sites.loc[x]['rhyp'], inplace=True)


# Rename column levels
df.rename_axis(["metric", "freq", "model"], axis=1, inplace=True)
df.head(2)


# Write to disk
%store df
write_pickle(df, "results/df.pickle")

Stored 'df' (DataFrame)
